In [4]:
#################################################
#    REAL TIME HAND GESTURE DETECTOR            #                        
#    by SUKANYA NASKAR                          #
#    Last Modified - 26th Dec, 2021             #
#################################################

#############################################################################################################
# Prerequisites ----                     
# Python(ver 3.9.0) + OpenCV(ver 4.5.4.60) + Mediapipe(ver 0.8.9) + Numpy(ver 1.20.2) + Tensorflow(ver 2.7.0)
#############################################################################################################

import numpy
import cv2
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model


## loading mediapipe model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# for webcam input
hands = mp_hands.Hands(model_complexity=0, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)


################ Load the gesture recognizer model
model = load_model('HandGesture_Model')

# Print types of gestures included in this project
with open('list_of_gestures.names', 'r') as f:
    gestures = f.read().split('\n')

    print(("\n\n" + str(gestures)) + "\n\n")


#### Write a loop to capture Camera input 

video_capture = cv2.VideoCapture(0)

while(True):
    if not video_capture.isOpened():
        print("\n\n ENABLE YOUR DEVICE WEBCAM AND START OVER.\n\n")
        break
    else:
        # Store the real time video capture and resize it in desired window size
        success, frame = video_capture.read()
        x, y, c = frame.shape
        # to flip the frame vertically so it takes a mirror image of the user's hand
        frame = cv2.flip(frame, 1)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
#         frame=cv2.resize(frame, (700, 500),interpolation = cv2.INTER_AREA)

        # Get hand landmark prediction
        result = hands.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        
        # Draw the hand annotations on the image.
        frame.flags.writeable = True
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        op_gesture = ''
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                landmarks = []
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS, mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())
                for lm in hand_landmarks.landmark:
                    lmx = (lm.x)
                    lmy = (lm.y)
                    landmarks.append([lmx, lmy])
                 
                print(landmarks)
                # Detect Gesture
                prediction = model.predict([landmarks])
                print(prediction)
                index = numpy.argmax(prediction)
                print(index)
                op_gesture = gestures[index]
                print(op_gesture)
                
        # show the prediction on the frame
        cv2.putText(frame, op_gesture, (10, 50), cv2.FONT_ITALIC, 1, (255,255,255), 2, cv2.LINE_AA)
                
        cv2.imshow("Let's Test Our Code", frame)
        # press 'x' to exit
        if cv2.waitKey(1) == ord('x'):
            break
        
# When we exit we have to stop the capture and destroy all ative windows
video_capture.release()
cv2.destroyAllWindows()
    



['call_me', 'fingers crossed', 'fist', 'okay', 'paper', 'peace', 'pointing up', 'rock on', 'thumbs up', 'victory']


[[0.13075754046440125, 0.6989262104034424], [0.15617281198501587, 0.6390239596366882], [0.19239482283592224, 0.590114951133728], [0.22324758768081665, 0.5650820732116699], [0.2480907440185547, 0.5505172610282898], [0.19732451438903809, 0.5534282326698303], [0.242531418800354, 0.5147765874862671], [0.27426671981811523, 0.5036839246749878], [0.298501193523407, 0.5011178851127625], [0.19874253869056702, 0.5690672993659973], [0.25115352869033813, 0.528555154800415], [0.28823235630989075, 0.5195410251617432], [0.3177889585494995, 0.5183572769165039], [0.2019471377134323, 0.5973218083381653], [0.255982369184494, 0.564091145992279], [0.2933412790298462, 0.5565909147262573], [0.3219184875488281, 0.5548642873764038], [0.20643959939479828, 0.6351625919342041], [0.24960492551326752, 0.6135039329528809], [0.27825263142585754, 0.6066025495529175], [0.30243200063705444, 0.6055877208

[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970166]]
4
paper
[[0.11342184245586395, 0.7371200919151306], [0.14510485529899597, 0.6908555030822754], [0.17560511827468872, 0.6480709910392761], [0.20228298008441925, 0.6215035915374756], [0.22602657973766327, 0.6031352281570435], [0.18283826112747192, 0.5964365601539612], [0.21628905832767487, 0.5401949882507324], [0.2418828010559082, 0.5136539340019226], [0.2622978985309601, 0.4981141686439514], [0.1901131272315979, 0.5989277362823486], [0.24582405388355255, 0.5435320734977722], [0.288313627243042, 0.5228641033172607], [0.3207469582557678, 0.5142983794212341], [0.19923684000968933, 0.6154413819313049], [0.2600715160369873, 0.5643940567970276], [0.3046838343143463, 0.5495041012763977], [0.33799657225608826, 0.5447615385055542], [0.21190407872200012, 0.6448323130607605], [0.2587639093399048, 0.6096331477165222], [0.288729727268219, 0.602850615978241], [0.31229162216186523, 0.605

[[0.11555025726556778, 0.7409698367118835], [0.14738664031028748, 0.6911033391952515], [0.1788092851638794, 0.6454862952232361], [0.20554912090301514, 0.618812620639801], [0.22879798710346222, 0.601632297039032], [0.18651391565799713, 0.5888103246688843], [0.22128015756607056, 0.5368794798851013], [0.24797673523426056, 0.5109302997589111], [0.2700522840023041, 0.49673402309417725], [0.19330564141273499, 0.5918832421302795], [0.24469438195228577, 0.5380212068557739], [0.2843610644340515, 0.5178433656692505], [0.3156778812408447, 0.5090532302856445], [0.20061244070529938, 0.6100494265556335], [0.2579583525657654, 0.5625567436218262], [0.30039942264556885, 0.5498039722442627], [0.33277684450149536, 0.5452561378479004], [0.21239729225635529, 0.6408523917198181], [0.257472962141037, 0.6076014041900635], [0.28639718890190125, 0.6001616716384888], [0.31022581458091736, 0.6003721356391907]]
[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0

[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970166]]
4
paper
[[0.1162230521440506, 0.7338961362838745], [0.14319539070129395, 0.6833899617195129], [0.1744748055934906, 0.6392198801040649], [0.20122592151165009, 0.6129900217056274], [0.2246955931186676, 0.5971095561981201], [0.1853264719247818, 0.5899847745895386], [0.21935386955738068, 0.5374820828437805], [0.24576829373836517, 0.5122852325439453], [0.2666186988353729, 0.49884331226348877], [0.19433212280273438, 0.5933676958084106], [0.24721062183380127, 0.5395604968070984], [0.28708788752555847, 0.5202012062072754], [0.317800372838974, 0.511602520942688], [0.20468424260616302, 0.610926628112793], [0.2631221413612366, 0.5616868734359741], [0.3055022358894348, 0.5474746227264404], [0.33666908740997314, 0.5428675413131714], [0.21788173913955688, 0.6409180164337158], [0.2634167969226837, 0.6064866185188293], [0.2915790379047394, 0.5990748405456543], [0.31340476870536804, 0.601

[[0.10998028516769409, 0.7438896298408508], [0.14216656982898712, 0.7007791996002197], [0.17358137667179108, 0.6589481234550476], [0.20097525417804718, 0.6335002779960632], [0.22502170503139496, 0.616919994354248], [0.1790420562028885, 0.5990393757820129], [0.21397019922733307, 0.5424060821533203], [0.2432214468717575, 0.5163166522979736], [0.26730209589004517, 0.5032352209091187], [0.18633325397968292, 0.5995739698410034], [0.2380913347005844, 0.5399917960166931], [0.2799364924430847, 0.5195857882499695], [0.31202957034111023, 0.5127347707748413], [0.1962689757347107, 0.6156694889068604], [0.25452178716659546, 0.56180739402771], [0.29891204833984375, 0.5467338562011719], [0.3315695524215698, 0.5427372455596924], [0.21118934452533722, 0.6454064846038818], [0.258533239364624, 0.609025776386261], [0.28927770256996155, 0.6012614369392395], [0.31348466873168945, 0.6034480333328247]]
[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.099

...

[[0.5614567399024963, 0.4777294099330902], [0.5394493937492371, 0.506931483745575], [0.5121612548828125, 0.5275917649269104], [0.48917004466056824, 0.5433449149131775], [0.46973174810409546, 0.536636471748352], [0.4787912368774414, 0.404446005821228], [0.44667577743530273, 0.49275586009025574], [0.46827593445777893, 0.5113656520843506], [0.48391398787498474, 0.49673330783843994], [0.4879002273082733, 0.3821067810058594], [0.452031672000885, 0.49661779403686523], [0.47338172793388367, 0.5067638158798218], [0.48229241371154785, 0.48180222511291504], [0.4994182586669922, 0.3877220153808594], [0.4798438549041748, 0.5003035664558411], [0.4972827732563019, 0.506687581539154], [0.5010731816291809, 0.483952134847641], [0.5106989741325378, 0.4067544639110565], [0.49747881293296814, 0.4973801076412201], [0.5141603946685791, 0.5075176358222961], [0.5230810642242432, 0.48773014545440674]]
[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970

[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970166]]
4
paper
[[0.5461562275886536, 0.4851211905479431], [0.5274235010147095, 0.5114984512329102], [0.5022836327552795, 0.5274515151977539], [0.48434212803840637, 0.5466034412384033], [0.47628524899482727, 0.5496466159820557], [0.4676547646522522, 0.4119131565093994], [0.4349285364151001, 0.4953484535217285], [0.45840293169021606, 0.512119710445404], [0.47212904691696167, 0.4908393323421478], [0.4775863289833069, 0.3913409411907196], [0.44077932834625244, 0.4952456057071686], [0.46984702348709106, 0.5063894391059875], [0.4793400168418884, 0.4729098975658417], [0.48913806676864624, 0.39641663432121277], [0.4649781286716461, 0.4996471405029297], [0.4878435730934143, 0.5052159428596497], [0.49422064423561096, 0.4719849228858948], [0.500405490398407, 0.4113253653049469], [0.4842894673347473, 0.4911348819732666], [0.501720666885376, 0.5001276135444641], [0.512236475944519, 0.4750556

[[0.5255431532859802, 0.4721073806285858], [0.5241167545318604, 0.5044064521789551], [0.5073030591011047, 0.5252441763877869], [0.48919928073883057, 0.5366251468658447], [0.4751839339733124, 0.5345789790153503], [0.471143901348114, 0.43391120433807373], [0.4435000717639923, 0.49425262212753296], [0.46446532011032104, 0.5071870684623718], [0.47696197032928467, 0.49593719840049744], [0.46565935015678406, 0.40467405319213867], [0.4394761621952057, 0.48236283659935], [0.46411287784576416, 0.4909588694572449], [0.4736837148666382, 0.47062793374061584], [0.4640248417854309, 0.39302122592926025], [0.44331008195877075, 0.4672228693962097], [0.4650973677635193, 0.4749216139316559], [0.47383859753608704, 0.45787087082862854], [0.4647144079208374, 0.39129453897476196], [0.4479442834854126, 0.44758954644203186], [0.4652702510356903, 0.4613635540008545], [0.4766637980937958, 0.4512762129306793]]
[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0

[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970166]]
4
paper
[[0.5469207763671875, 0.48445063829421997], [0.5222564339637756, 0.5027926564216614], [0.4967687129974365, 0.5151066184043884], [0.4814579486846924, 0.5313272476196289], [0.47364309430122375, 0.5386237502098083], [0.45784059166908264, 0.412328839302063], [0.42407843470573425, 0.4838256537914276], [0.44781777262687683, 0.5002250671386719], [0.4653834104537964, 0.4872073531150818], [0.47477057576179504, 0.3964788317680359], [0.4428898096084595, 0.4882466197013855], [0.4689006209373474, 0.4956243932247162], [0.47746574878692627, 0.4683542549610138], [0.4934934675693512, 0.4025960862636566], [0.4732632637023926, 0.49449482560157776], [0.49452632665634155, 0.4971906542778015], [0.5002322196960449, 0.4699658155441284], [0.5121408700942993, 0.4200012683868408], [0.5001601576805115, 0.49147605895996094], [0.5161911249160767, 0.4979736804962158], [0.5248766541481018, 0.477

[[0.5398244261741638, 0.5468539595603943], [0.5134202241897583, 0.5689330697059631], [0.4801376461982727, 0.574985146522522], [0.46226251125335693, 0.5559802651405334], [0.4584515690803528, 0.5282629728317261], [0.45780420303344727, 0.5175315141677856], [0.4384937286376953, 0.5079865455627441], [0.4294494688510895, 0.5094473958015442], [0.4268375635147095, 0.5171074867248535], [0.4699670374393463, 0.4888630509376526], [0.4521723687648773, 0.46894577145576477], [0.4446342885494232, 0.47276729345321655], [0.4404701292514801, 0.4817127585411072], [0.4881041944026947, 0.4690253734588623], [0.47417309880256653, 0.453883558511734], [0.4684095084667206, 0.45922160148620605], [0.465242475271225, 0.46833276748657227], [0.5118414163589478, 0.4551425576210022], [0.5002671480178833, 0.4475666880607605], [0.4954231381416321, 0.4565623104572296], [0.4927103519439697, 0.4662879407405853]]
[[0.10018963 0.09975885 0.09997687 0.10013349 0.10040431 0.10038239
  0.0996008  0.10010444 0.09974758 0.09970166